In [1]:
import igl
import math
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import os
root_folder = os.getcwd()

In [2]:
v, f = igl.read_triangle_mesh(os.path.join(root_folder, "oblate.off"))
global Kb
global Kv
global Ka
Ka=0.1
Kv= 0.02
Kb=8.22e-5 
Kt= 0.7 * 3.14 * 4 / 3

In [3]:
def adjacent_face(v,f):
    df=pd.DataFrame(f,columns=list('ABC'))
    row_numbers=[]
    for i in range(len(v)):
        row_numbers.append((df.index[(df['A'] == i)|(df['B'] == i) | (df['C'] == i)].tolist()))
    return row_numbers

adjacent_faces=adjacent_face(v,f)
k=adjacent_faces[0][2]
n = igl.per_vertex_normals(v, f)
face_normal=igl.per_face_normals(v,f,n)
face_normal[k]

In [4]:
def cal_Area(v,f):
    dbl_area = igl.doublearea(v, f)    
    Area=np.sum(dbl_area)/2
    return Area

In [5]:
def volgrad(v,f):
    n = igl.per_vertex_normals(v, f)
    face_normal=igl.per_face_normals(v,f,n)
    dbl_area = igl.doublearea(v, f)/2
    adjacent_vertices=igl.adjacency_list(f)
    adjacent_faces=adjacent_face(v,f)
    volumegrad=[]
    for i in range(len(v)):
        vol_ij=0
        for j in range(len(adjacent_faces[i])):
            k=adjacent_faces[i][j]
            Area=dbl_area[k]
            FaceNorm=face_normal[k]
            vol_ij=vol_ij+((1/3)*Area*FaceNorm)
        volumegrad.append(vol_ij)
    volumegrad=np.array(volumegrad)
    return volumegrad

In [6]:
def areaGrad(v,f):
    n=igl.per_vertex_normals(v,f)
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    l = igl.cotmatrix(v, f) ###laplacian-operator
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    hn = -minv.dot(l.dot(v))
    h_mean = np.linalg.norm(hn, axis=1)
    areagrad=2*n*h_mean[:,None]
    return areagrad

In [7]:
def cal_volume(v,f):
    Volume=0
    for i in range(len(f)):
        sum=0
        p0x=v[f[i][0]][0]
        p0y=v[f[i][0]][1]
        p0z=v[f[i][0]][2]
        p1x=v[f[i][1]][0]
        p1y=v[f[i][1]][1]
        p1z=v[f[i][1]][2]
        p2x=v[f[i][2]][0]
        p2y=v[f[i][2]][1]
        p2z=v[f[i][2]][2]
        v321= p2x*p1y*p0z
        v231= p1x*p2y*p0z
        v312= p2x*p0y*p1z
        v132= p0x*p2y*p1z
        v213= p1x*p0y*p2z
        v123= p0x*p1y*p2z
        sum=(-v321+ v231+ v312-v132-v213+ v123) / 6.0
        #print(sum)
        Volume=Volume+sum
    return Volume

def fun_volgrad(v,f):
    n = igl.per_vertex_normals(v, f)
    #vector=np.array([1,1,1])
    #norm=vector/np.linalg.norm(vector)

    face_normal=igl.per_face_normals(v,f,n)
    dbl_area = igl.doublearea(v, f)/2
    volume_grad=face_normal*dbl_area[:,None]
    
    return volume_grad/3

In [8]:
def fun_ForceDensity(v,f):
    k = igl.gaussian_curvature(v, f)
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    kn = minv.dot(k) 
    area_voronoi=m.diagonal()
    l = igl.cotmatrix(v, f) ###laplacian-operator
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    hn = -minv.dot(l.dot(v))
    h_mean = np.linalg.norm(hn, axis=1)
    Lap_H=minv.dot(l.dot(h_mean/2))
    first_product=h_mean*(((h_mean/2)**2)-k)
    second_product=np.add(Lap_H,first_product)
    n = igl.per_vertex_normals(v, f)
    #normal_v=n/np.linalg.norm(n)
    ### Force Density and Nodal Force
    Kb=1
    #Force_density=[] # Force_Density
    Force_Nodal=[]

    #for i in range (len(v)):
    Force_density=2*Kb*(n*second_product[:,None])
        #Force_Nodal.append(Force_density[i]*area_voronoi[i])
        #print(Force_density[i]) 
        #print(Force_Nodal[i])
        
    return Force_density

In [9]:
def Force_Area(Area_old,grad_Area,Area_new):
    #grad_Area=volgrad(v,f)
    #Area_old=cal_volume(v,f)
    #Area_new=cal_Area(pos_new,f)
    Force_Area=-2*Ka*((Area_new-Area_old)/Area_old)*(grad_Area)
    return Force_Area

In [10]:
def Force_Volume(volume_old,grad_Volume,volume_new):
    #grad_Volume=volgrad(v,f)
    #Volume_old=cal_volume(v,f)
    #Volume_new=cal_volume(pos_new,f)
    Force_Volume=-2*(Kv+Kt)*((volume_new-volume_old)/volume_old)*grad_Volume
    return Force_Volume

In [11]:
volume_old=cal_volume(v,f)
Area_old=cal_Area(v,f)
grad_Area=areaGrad(v,f)
grad_Volume=volgrad(v,f)
Area_new=cal_Area(v,f)
volume_new=cal_volume(v,f)

In [12]:
FB=np.array(fun_ForceDensity(v,f))
FA=Force_Area(Area_old,grad_Area,Area_new)
FV=Force_Volume(volume_old,grad_Volume,volume_new)
def Total_Force(FB,FA,FV):
    Total_force=(FB+FA+FV)
    return Total_force

In [13]:
TF=Total_Force(FB,FA,FV)

grad_A=areaGrad(v,f)

grad_Volume=volgrad(v,f)
grad_Volume=np.array(grad_Volume)
grad_Volume.shape

In [14]:
###Velocity Verlet Main 
dt=0.01
hdt=0.5*dt
iterations =1000

vel_init=np.zeros((len(v),3));
vel_new_half=np.zeros((len(v),3))
vel_new=np.zeros((len(v),3))
pos_new=np.zeros((len(v),3))



for i in range(iterations):
    for j in range(len(v)):
        vel_new_half[j]=np.add(vel_init[j],(0.5*TF[j]*dt))
        
        pos_new[j]=np.add(v[j],(vel_new_half[j]*dt))
        #print(pos_new)
    FB_new=np.array(fun_ForceDensity(pos_new,f)) ##bending_force
    Area_new=cal_Area(pos_new,f)
    grad_Area_new=volgrad(pos_new,f)
    volume_new=cal_volume(pos_new,f)
    grad_Volume_new=volgrad(pos_new,f)
    FB_new=np.array(fun_ForceDensity(v,f))
    FA_new=Force_Area(Area_old,grad_Area_new,Area_new)
    FV_new=Force_Volume(volume_old,grad_Volume_new,volume_new)
    TF_new=Total_Force(FB_new,FA_new,FV_new)
    
    
    #print(totalpressure_new)
    for k in range(len(v)):
        vel_new[k]=np.add(vel_new_half[k],(0.5*TF_new[k]*dt))
        #print(vel_new)
        vel_init[k]=vel_new[k]
        TF[k]=TF_new[k]

In [15]:
pos_new[1]-v[1]

array([-0.0067988 , -0.01520982, -0.03942173])

In [16]:
plot(pos_new,f, shading={"wireframe": True}, return_plot=True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0084348…